In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#Data Transform
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [13]:
#Defining Hyperparameters
input_size = 784
hidden_class = 1000
num_classes = 10
num_epoch = 50
learning_rate = 0.001
batch_size = 10

In [5]:
#Importing data and Preprocessing it
train_dataset = datasets.CIFAR10(root = './data', train=True, transform= transform, download=True)
test_dataset = datasets.CIFAR10(root = './data', train=False, transform= transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
#Dataloader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [7]:
#Classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [15]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x


In [16]:
#Creating the Mode
model = ConvolutionalNeuralNetwork().to(device)

In [17]:
#Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [18]:
#Training the Model

n_total_steps = len(train_loader)
for ep in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        #Forward Pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        #Backward Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Printing output
        if (i + 1) % 1000 == 0:
            print(f'Epoch: [{ep + 1} / {num_epoch}], Step: [{i + 1} / {n_total_steps}], Loss: {loss.item():.4f}')

Epoch: [1 / 50], Step: [1000 / 5000], Loss: 2.2680
Epoch: [1 / 50], Step: [2000 / 5000], Loss: 2.3139
Epoch: [1 / 50], Step: [3000 / 5000], Loss: 2.3083
Epoch: [1 / 50], Step: [4000 / 5000], Loss: 2.2858
Epoch: [1 / 50], Step: [5000 / 5000], Loss: 2.3011
Epoch: [2 / 50], Step: [1000 / 5000], Loss: 2.2879
Epoch: [2 / 50], Step: [2000 / 5000], Loss: 2.2824
Epoch: [2 / 50], Step: [3000 / 5000], Loss: 2.3012
Epoch: [2 / 50], Step: [4000 / 5000], Loss: 2.3610
Epoch: [2 / 50], Step: [5000 / 5000], Loss: 2.1306
Epoch: [3 / 50], Step: [1000 / 5000], Loss: 2.1994
Epoch: [3 / 50], Step: [2000 / 5000], Loss: 2.0324
Epoch: [3 / 50], Step: [3000 / 5000], Loss: 1.9771
Epoch: [3 / 50], Step: [4000 / 5000], Loss: 2.3008
Epoch: [3 / 50], Step: [5000 / 5000], Loss: 1.8430
Epoch: [4 / 50], Step: [1000 / 5000], Loss: 2.4965
Epoch: [4 / 50], Step: [2000 / 5000], Loss: 1.7553
Epoch: [4 / 50], Step: [3000 / 5000], Loss: 1.7694
Epoch: [4 / 50], Step: [4000 / 5000], Loss: 1.8228
Epoch: [4 / 50], Step: [5000 / 

In [19]:
#Checking Accuracy

with torch.no_grad():
    correct = 0
    samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        samples += labels.size(0)
        correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    acc = 100 * correct / samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]} : {acc} %')

Accuracy of the network: 63.03 %
Accuracy of plane : 62.9 %
Accuracy of car : 70.7 %
Accuracy of bird : 54.6 %
Accuracy of cat : 40.5 %
Accuracy of deer : 61.4 %
Accuracy of dog : 38.2 %
Accuracy of frog : 71.7 %
Accuracy of horse : 82.0 %
Accuracy of ship : 77.0 %
Accuracy of truck : 71.3 %
